In [1]:
#import sys, os
#sys.path.insert(0, os.path.dirname(os.path.abspath('../desdeo_emo')))

In [2]:
import plotly.graph_objects as go
import numpy as np
import pandas as pd

from desdeo_problem.Variable import variable_builder
from desdeo_problem.Objective import _ScalarObjective
from desdeo_problem.Problem import MOProblem
from desdeo_problem.testproblems.TestProblems import test_problem_builder

from desdeo_emo.EAs.NSGAIII import NSGAIII
from desdeo_emo.EAs.RVEA import RVEA
from desdeo_emo.othertools.plotlyanimate import animate_init_, animate_next_

## Coello MOP7

### Definition
![Definition](https://raw.githubusercontent.com/industrial-optimization-group/desdeo-emo/master/examples/notebook_extras/MOP7.png)

### Pareto set and front
![Front](https://raw.githubusercontent.com/industrial-optimization-group/desdeo-emo/master/examples/notebook_extras/MOP7_PF.png)

## Define objective functions

In [3]:
def f_1(x):
    term1 = ((x[:,0] - 2) ** 2) / 2
    term2 = ((x[:,1] + 1) ** 2) / 13
    return term1 + term2 + 3

def f_2(x):
    term1 = ((x[:, 0] + x[:, 1] - 3) ** 2) / 36
    term2 = ((-x[:, 0] + x[:, 1] + 2) ** 2) / 8
    return term1 + term2 - 17

def f_3(x):
    term1 = ((x[:, 0] + (2 * x[:, 1]) - 1) ** 2) / 175
    term2 = ((-x[:, 0] + 2* x[:, 1]) ** 2) / 17
    return term1 + term2 - 13

Note that the expected input `x` is two dimensional. It should be a 2-D numpy array.

## Create Variable objects

In [4]:
help(variable_builder)

Help on function variable_builder in module desdeo_problem.Variable:

variable_builder(names: List[str], initial_values: Union[List[float], numpy.ndarray], lower_bounds: Union[List[float], numpy.ndarray] = None, upper_bounds: Union[List[float], numpy.ndarray] = None) -> List[desdeo_problem.Variable.Variable]
    Automatically build all variable objects.
    
    Args:
        names (List[str]): Names of the variables
        initial_values (np.ndarray): Initial values taken by the variables.
        lower_bounds (Union[List[float], np.ndarray], optional): Lower bounds of the
            variables. If None, it defaults to negative infinity. Defaults to None.
        upper_bounds (Union[List[float], np.ndarray], optional): Upper bounds of the
            variables. If None, it defaults to positive infinity. Defaults to None.
    
    Raises:
        VariableError: Lengths of the input arrays are different.
    
    Returns:
        List[Variable]: List of variable objects



In [5]:
list_vars = variable_builder(['x', 'y'],
                             initial_values = [0,0],
                             lower_bounds=[-400, -400],
                             upper_bounds=[400, 400])
list_vars

## Create Objective objects

In [6]:
f1 = _ScalarObjective(name='f1', evaluator=f_1)
f2 = _ScalarObjective(name='f2', evaluator=f_2)
f3 = _ScalarObjective(name='f3', evaluator=f_3)
list_objs = [f1, f2, f3]

## Create the problem object

In [7]:
problem = MOProblem(variables=list_vars, objectives=list_objs)

## Using the EAs

Pass the problem object to the EA, pass parameters as arguments if required.

In [8]:
help(RVEA)

Help on class RVEA in module desdeo_emo.EAs.RVEA:

class RVEA(desdeo_emo.EAs.BaseEA.BaseDecompositionEA)
 |  RVEA(problem: desdeo_problem.Problem.MOProblem, population_size: int = None, population_params: Dict = None, initial_population: desdeo_emo.population.Population.Population = None, alpha: float = 2, lattice_resolution: int = None, selection_type: str = None, a_priori: bool = False, interact: bool = False, use_surrogates: bool = False, n_iterations: int = 10, n_gen_per_iter: int = 100, total_function_evaluations: int = 0, time_penalty_component: Union[str, float] = None)
 |  
 |  The python version reference vector guided evolutionary algorithm.
 |  
 |  Most of the relevant code is contained in the super class. This class just assigns
 |  the APD selection operator to BaseDecompositionEA.
 |  
 |  NOTE: The APD function had to be slightly modified to accomodate for the fact that
 |  this version of the algorithm is interactive, and does not have a set termination
 |  criteria. T

In [9]:
evolver = NSGAIII(problem,
                  n_iterations=3,
                  n_gen_per_iter=20,
                  population_size=500)

In [10]:
plot, pref = evolver.requests()

In [11]:
pref

In [12]:
while evolver.continue_evolution():
    evolver.iterate()

## Extracting optimized decision variables and objective values

In [13]:
individuals = evolver.population.individuals

fig1 = go.Figure(data=go.Scatter(x=individuals[:,0], y=individuals[:,1], mode="markers"))
fig1

In [14]:
objectives = evolver.population.objectives
fig2 = go.Figure(data=go.Scatter3d(x=objectives[:,0],
                                   y=objectives[:,1],
                                   z=objectives[:,2],
                                   mode="markers",
                                   marker_size=1))
fig2

## Coello MOP5

### Definition
![Definition](https://raw.githubusercontent.com/industrial-optimization-group/desdeo-emo/master/examples/notebook_extras/MOP5.png)

### Pareto set and front
![Front](https://raw.githubusercontent.com/industrial-optimization-group/desdeo-emo/master/examples/notebook_extras/MOP5_PF.png)

In [15]:
def f_4(x):
    term = x[:, 0]**2 + x[:, 1]**2
    return 0.5*term + np.sin(term)

def f_5(x):
    term1 = ((3*x[:, 0] - 2*x[:,1] + 4)**2)/8
    term2 = ((x[:, 0] - x[:,1] + 1)**2)/27
    return term1 + term2 + 15

def f_6(x):
    term = x[:, 0]**2 + x[:, 1]**2
    return (1/(term + 1)) - 1.1 * np.exp(-term)

In [16]:
list_vars = variable_builder(['x', 'y'],
                             initial_values = [0,0],
                             lower_bounds=[-30, -30],
                             upper_bounds=[30, 30])

In [17]:
f1 = _ScalarObjective(name='f1', evaluator=f_4)
f2 = _ScalarObjective(name='f2', evaluator=f_5)
f3 = _ScalarObjective(name='f3', evaluator=f_6)

In [18]:
problem = MOProblem(variables=list_vars, objectives=[f1, f2, f3])

In [19]:
evolver = RVEA(problem)

In [20]:
figure = animate_init_(evolver.population.objectives, filename="MOP5.html")

Plot saved as:  MOP5.html
View the plot by opening the file in browser.
To view the plot in Jupyter Notebook, use the IFrame command.


### Refresh the plot page after each iteration to get the updated animation

In [21]:
while evolver.continue_evolution():
    evolver.iterate()
    print(evolver.selection_operator.time_penalty_function())
    figure = animate_next_(
        evolver.population.objectives,
        figure,
        filename="MOP5.html",
        generation=evolver._iteration_counter,
    )

0.1
0.2
0.3
0.4
0.5
0.6
0.7
0.8
0.9
1.0


## Interaction in EAs

The reference point method has been implemented.

In [22]:
help(test_problem_builder)

Help on function test_problem_builder in module desdeo_problem.testproblems.TestProblems:

test_problem_builder(name: str, n_of_variables: int = None, n_of_objectives: int = None) -> desdeo_problem.Problem.MOProblem
    Build test problems. Currently supported: ZDT1-4, ZDT6, and DTLZ1-7.
    
    Args:
        name (str): Name of the problem in all caps. For example: "ZDT1", "DTLZ4", etc.
        n_of_variables (int, optional): Number of variables. Required for DTLZ problems,
            but can be skipped for ZDT problems as they only support one variable value.
        n_of_objectives (int, optional): Required for DTLZ problems,
            but can be skipped for ZDT problems as they only support one variable value.
    
    Raises:
        ProblemError: When one of many issues occur while building the MOProblem
            instance.
    
    Returns:
        MOProblem: The test problem object



In [23]:
problem = test_problem_builder(name="DTLZ1", n_of_variables=30, n_of_objectives=3)

In [31]:
evolver = NSGAIII(problem, interact=True, n_iterations=5, n_gen_per_iter=400)
figure = animate_init_(evolver.population.objectives, filename="dtlz1.html")

Plot saved as:  dtlz1.html
View the plot by opening the file in browser.
To view the plot in Jupyter Notebook, use the IFrame command.


In [32]:
plot, pref = evolver.requests()

### Loop over the following three cells, updating the preferences as desired

Try changing the preference once the Pareto front has been reached.
Note the format of providing the preference.
Check out the content of pref and plot

In [33]:
print(pref.content['message'])

Provide a reference point worse than or equal to the ideal point:
f1    0.28408
f2    1.13228
f3    14.9005
Name: ideal, dtype: object
The reference point will be used to focus the reference vectors towards the preferred region.
If a reference point is not provided, the previous state of the reference vectors is used.
If the reference point is the same as the ideal point, the ideal point, the reference vectors are spread uniformly in the objective space.


In [37]:
response = evolver.population.ideal_fitness_val + [0.5,0.7,0.1]
pref.response = pd.DataFrame([response], columns=pref.content['dimensions_data'].columns)

In [38]:
plot, pref = evolver.iterate(pref)
figure = animate_next_(
    plot.content['data'].values,
    figure,
    filename="dtlz1.html",
    generation=evolver._iteration_counter,
)

message = (f"Current generation number:{evolver._current_gen_count}. "
           f"Is looping back recommended: {'Yes' if evolver.continue_evolution() else 'No'}")
print(message)

Current generation number:400. Is looping back recommended: Yes
